<a href="https://colab.research.google.com/github/ittekiou/EgQE/blob/main/MMZW_03b_Phase_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# MMZW-03b Phase 1 (U1 shared-branch) — 12 cases
# Outputs:
#   - mmzw03b_phase1_results.csv
#   - mmzw03b_phase1_results.jsonl
#   - (optional) per-case plot PNGs if SAVE_PLOTS=True

import mpmath as mp
import numpy as np
import pandas as pd
import json, os
import matplotlib.pyplot as plt

mp.mp.dps = 30
TAU = 2*np.pi

SAVE_PLOTS = False
PLOT_DIR = "mmzw03b_plots"
if SAVE_PLOTS:
    os.makedirs(PLOT_DIR, exist_ok=True)

# ---------- core probe ----------
def zeta_arg_vec(ts: np.ndarray) -> np.ndarray:
    return np.array([float(mp.arg(mp.zeta(0.5 + 1j*float(t)))) for t in ts], dtype=float)

def unwrap_and_align(u_base: np.ndarray, u_shift: np.ndarray) -> np.ndarray:
    """U1 shared-branch alignment: same unwrap + global 2πk alignment"""
    diff_med = np.median(u_base - u_shift)
    k = int(np.round(diff_med / TAU))
    u2 = u_shift + k*TAU
    diff_med2 = np.median(u_base - u2)
    k2 = int(np.round(diff_med2 / TAU))
    return u2 + k2*TAU

def compute_Dd(ts: np.ndarray, delta: float) -> np.ndarray:
    phi0 = zeta_arg_vec(ts)
    u0 = np.unwrap(phi0, discont=np.pi)

    phi1 = zeta_arg_vec(ts + delta)
    u1 = np.unwrap(phi1, discont=np.pi)

    u1a = unwrap_and_align(u0, u1)  # U1
    return (u1a - u0) / delta

# ---------- metrics ----------
def spike_frac_mad10(D: np.ndarray) -> float:
    med = np.median(D)
    mad = np.median(np.abs(D - med)) + 1e-12
    return float(np.mean(np.abs(D - med) > 10*mad))

def metrics(D: np.ndarray) -> dict:
    return {
        "spike_frac_MAD10": spike_frac_mad10(D),
        "abs_q90": float(np.quantile(np.abs(D), 0.90)),
        "abs_q99": float(np.quantile(np.abs(D), 0.99)),
        "max_abs": float(np.max(np.abs(D))),
        "energy_mean_D2": float(np.mean(D**2)),
    }

# ---------- regime label (rough, adjustable) ----------
def label_regime(m: dict) -> str:
    # very simple first-pass labels for the map
    if m["spike_frac_MAD10"] < 0.001 and m["abs_q99"] < 3 and m["max_abs"] < 50:
        return "F0_quiet"
    if m["spike_frac_MAD10"] >= 0.01:
        return "F1_dense_spikes"
    if m["spike_frac_MAD10"] < 0.01 and m["max_abs"] >= 100:
        return "F2_sparse_extremes"
    return "F?_mixed"

# ---------- Phase 1 grid (U1 only) ----------
cases = [
    # Band, N fixed to match 03a style
    {"case": "01", "band": (10, 60),   "N": 320, "delta": 0.1,   "U": "U1"},
    {"case": "02", "band": (10, 60),   "N": 320, "delta": 0.05,  "U": "U1"},
    {"case": "03", "band": (10, 60),   "N": 320, "delta": 0.01,  "U": "U1"},
    {"case": "04", "band": (10, 60),   "N": 320, "delta": 0.005, "U": "U1"},

    {"case": "05", "band": (200, 260), "N": 320, "delta": 0.1,   "U": "U1"},
    {"case": "06", "band": (200, 260), "N": 320, "delta": 0.05,  "U": "U1"},
    {"case": "07", "band": (200, 260), "N": 320, "delta": 0.01,  "U": "U1"},
    {"case": "08", "band": (200, 260), "N": 320, "delta": 0.005, "U": "U1"},

    {"case": "09", "band": (200, 300), "N": 480, "delta": 0.1,   "U": "U1"},
    {"case": "10", "band": (200, 300), "N": 480, "delta": 0.05,  "U": "U1"},
    {"case": "11", "band": (200, 300), "N": 480, "delta": 0.01,  "U": "U1"},
    {"case": "12", "band": (200, 300), "N": 480, "delta": 0.005, "U": "U1"},
]

rows = []
jsonl_path = "mmzw03b_phase1_results.jsonl"
open(jsonl_path, "w").close()  # reset

for c in cases:
    T1, T2 = c["band"]
    N = c["N"]
    delta = float(c["delta"])
    ts = np.linspace(T1, T2, N)
    dt = float((T2 - T1) / (N - 1))

    D = compute_Dd(ts, delta)
    m = metrics(D)
    reg = label_regime(m)

    row = {
        "case": c["case"],
        "band": f"[{T1},{T2}]",
        "delta": delta,
        "U": c["U"],
        "N": N,
        "dt": dt,
        **m,
        "regime": reg,
    }
    rows.append(row)

    with open(jsonl_path, "a") as f:
        f.write(json.dumps(row, ensure_ascii=False) + "\n")

    if SAVE_PLOTS:
        plt.figure()
        plt.plot(ts, D)
        plt.title(f"MMZW-03b P1 Case {c['case']}  Band[{T1},{T2}]  Δ={delta}  U1")
        plt.xlabel("t"); plt.ylabel("DΔ arg ζ")
        plt.tight_layout()
        plt.savefig(os.path.join(PLOT_DIR, f"case_{c['case']}.png"), dpi=160)
        plt.close()

df = pd.DataFrame(rows)
df.to_csv("mmzw03b_phase1_results.csv", index=False)

# A compact “map table” view (Band × Δ)
pivot = df.pivot_table(index="band", columns="delta", values="regime", aggfunc="first")
print("=== Phase 1 Results (summary) ===")
print(df[["case","band","delta","U","N","dt","spike_frac_MAD10","abs_q99","max_abs","energy_mean_D2","regime"]])
print("\n=== Regime Map (Band × Δ) ===")
print(pivot)


=== Phase 1 Results (summary) ===
   case       band  delta   U    N        dt  spike_frac_MAD10    abs_q99  \
0    01    [10,60]  0.100  U1  320  0.156740          0.028125  30.566986   
1    02    [10,60]  0.050  U1  320  0.156740          0.009375   1.128188   
2    03    [10,60]  0.010  U1  320  0.156740          0.000000   1.124085   
3    04    [10,60]  0.005  U1  320  0.156740          0.000000   1.124064   
4    05  [200,260]  0.100  U1  320  0.188088          0.056250  29.655996   
5    06  [200,260]  0.050  U1  320  0.188088          0.037500  61.047928   
6    07  [200,260]  0.010  U1  320  0.188088          0.006250   1.860981   
7    08  [200,260]  0.005  U1  320  0.188088          0.000000   1.860252   
8    09  [200,300]  0.100  U1  480  0.208768          0.066667  29.624644   
9    10  [200,300]  0.050  U1  480  0.208768          0.039583  61.002594   
10   11  [200,300]  0.010  U1  480  0.208768          0.004167   1.931989   
11   12  [200,300]  0.005  U1  480  0.2087